In [2]:
import pandas as pd
import sqlite3 as sq3
import datetime
import math

In [3]:
conn = sq3.connect('/home/usr1/bghaem/mu/proj/dlrm/support/data/avazu.db')
c = conn.cursor()
try:
    c.execute('''
                CREATE TABLE IF NOT EXISTS data
                (id integer,
                click integer,
                hour integer,
                C1 integer,
                banner_pos integer,
                site_id integer,
                site_domain integer,
                site_category integer,
                app_id integer,
                app_domain integer,
                app_category integer,
                device_id integer,
                device_ip integer,
                device_model integer,
                device_type integer,
                device_conn_type integer,
                C14 integer,
                C15 integer,
                C16 integer,
                C17 integer,
                C18 integer,
                C19 integer,
                C20 integer,
                C21 integer)
        ''')
    c.execute('''
                CREATE TABLE IF NOT EXISTS data_cleaned
                (id integer,
                click integer,
                hour integer,
                C1 integer,
                banner_pos integer,
                site_id integer,
                site_domain integer,
                site_category integer,
                app_id integer,
                app_domain integer,
                app_category integer,
                device_id integer,
                device_ip integer,
                device_model integer,
                device_type integer,
                device_conn_type integer,
                C14 integer,
                C15 integer,
                C16 integer,
                C17 integer,
                C18 integer,
                C19 integer,
                C20 integer,
                C21 integer)
        ''')
    c.execute('''
                CREATE TABLE IF NOT EXISTS col_counts
                (cat text,
                count integer)
            ''')
    conn.commit()
except sq3.OperationalError as e:
    print("Create failed", e)

In [97]:
setup = True #safety to avoid overwriting database all the time
if (not setup):
    with open('/home/usr1/bghaem/mu/proj/dlrm/support/data/avazu_train.csv', 'r') as f:

        print("resetting data table...")

        c.execute('''DELETE FROM data''')
        conn.commit()

        labels = f.readline().split(',')
        table_types = [lab + " integer" for lab in labels]
        print(table_types)

        print(f.readline())

        buffer = []
        counter = 1
        for line in f:
            items = line.split(',')
            conv_items = []
            for i, v in enumerate(items):
                if i >=0 and i < 5:
                    conv_items.append(v)
                elif i > 13:
                    conv_items.append(int(v))
                else:
                    conv_items.append(v)

            if len(conv_items) == 23:
                conv_items.insert(1,"1")

            if (len(buffer) < 100000):
                buffer.append(conv_items)
            else:
                print("loaded " + str(counter * 100000))
                counter = counter + 1
                c.executemany('INSERT INTO data VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', buffer)
                buffer = []
        
        conn.commit()

resetting data table...
['id integer', 'click integer', 'hour integer', 'C1 integer', 'banner_pos integer', 'site_id integer', 'site_domain integer', 'site_category integer', 'app_id integer', 'app_domain integer', 'app_category integer', 'device_id integer', 'device_ip integer', 'device_model integer', 'device_type integer', 'device_conn_type integer', 'C14 integer', 'C15 integer', 'C16 integer', 'C17 integer', 'C18 integer', 'C19 integer', 'C20 integer', 'C21\n integer']
1000009418151094273,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,ddd2926e,44956a24,1,2,15706,320,50,1722,0,35,-1,79

loaded 100000
loaded 200000
loaded 300000
loaded 400000
loaded 500000
loaded 600000
loaded 700000
loaded 800000
loaded 900000
loaded 1000000
loaded 1100000
loaded 1200000
loaded 1300000
loaded 1400000
loaded 1500000
loaded 1600000
loaded 1700000
loaded 1800000
loaded 1900000
loaded 2000000
loaded 2100000
loaded 2200000
loaded 2300000
loaded 2400000
loaded 2500000
loa

In [98]:
cs = """C1 integer,
banner_pos integer,
site_id integer,
site_domain integer,
site_category integer,
app_id integer,
app_domain integer,
app_category integer,
device_id integer,
device_ip integer,
device_model integer,
device_type integer,
device_conn_type integer,
C14 integer,
C15 integer,
C16 integer,
C17 integer,
C18 integer,
C19 integer,
C20 integer,
C21 integer"""
cat_dicts = {}
for line in cs.split("\n"):
    cc, i = line.split(" ") 
    uniq_cid = c.execute(f"SELECT DISTINCT {cc} FROM data")
    cat_dicts[cc]={}
    cat_dicts[cc][-1] = 0
    counter = 1
    print("processing", cc,end='')
    for item in uniq_cid:
        if (item != -1):
            cat_dicts[cc][item] = counter
            counter = counter + 1
            if (counter % 5000 == 0):
                print(".",end='')
    print("")

processing C1
processing banner_pos
processing site_id
processing site_domain.
processing site_category
processing app_id.
processing app_domain
processing app_category
processing device_id....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
processing device_ip..................................................................................................................................................................................................................................................................

In [99]:
uniq_convert_setup = False
if (not uniq_convert_setup):

        print("resetting cleaned data table...")

        c.execute('''DELETE FROM data_cleaned''')
        conn.commit()

        col_names = ['C1',
                     'banner_pos',
                     'site_id',
                     'site_domain',
                     'site_category',
                     'app_id',
                     'app_domain',
                     'app_category',
                     'device_id',
                     'device_ip',
                     'device_model',
                     'device_type',
                     'device_conn_type',
                     'C14',
                     'C15',
                     'C16',
                     'C17',
                     'C18',
                     'C19',
                     'C20',
                     'C21']
       
      
        off = 0
        batch_size = 1000000
        while (off < 50000000):
            sel_rows = c.execute(f"SELECT * FROM data LIMIT {batch_size} OFFSET {off}")
            buffer = []
            for rr, row in enumerate(sel_rows):
                tmp_row = []
                for ii, item in enumerate(row):
                    if ii < 2: #id, click, hour -- do nothing
                        tmp_row.append(item)
                    elif ii == 2:
                        tmp_row.append(math.log(datetime.datetime.utcfromtimestamp(item).hour + 1))
                    else:
                        if item == -1:
                            tmp_row.append(0)
                        else:
                            tmp_row.append(cat_dicts[col_names[ii-3]][(item,)])

                buffer.append(tmp_row)
               
            print("cleaned " + str(len(buffer)), off)
            if (len(buffer) == 0):
                #break out early if we have overshot
                break
            c.executemany('INSERT INTO data_cleaned VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', buffer)
            buffer = []
            off = off + batch_size
        
        conn.commit()

resetting cleaned data table...
cleaned 1000000 0
cleaned 1000000 1000000
cleaned 1000000 2000000
cleaned 1000000 3000000
cleaned 1000000 4000000
cleaned 1000000 5000000
cleaned 1000000 6000000
cleaned 1000000 7000000
cleaned 1000000 8000000
cleaned 1000000 9000000
cleaned 1000000 10000000
cleaned 1000000 11000000
cleaned 1000000 12000000
cleaned 1000000 13000000
cleaned 1000000 14000000
cleaned 1000000 15000000
cleaned 1000000 16000000
cleaned 1000000 17000000
cleaned 1000000 18000000
cleaned 1000000 19000000
cleaned 1000000 20000000
cleaned 1000000 21000000
cleaned 1000000 22000000
cleaned 1000000 23000000
cleaned 1000000 24000000
cleaned 1000000 25000000
cleaned 1000000 26000000
cleaned 1000000 27000000
cleaned 1000000 28000000
cleaned 1000000 29000000
cleaned 1000000 30000000
cleaned 1000000 31000000
cleaned 1000000 32000000
cleaned 1000000 33000000
cleaned 1000000 34000000
cleaned 1000000 35000000
cleaned 1000000 36000000
cleaned 1000000 37000000
cleaned 1000000 38000000
cleaned 1

In [5]:
#counts = [(m[0],len(m[1].keys())) for m in cat_dicts.items()]
counts_setup = True

if (not counts_setup):
   
    print("resetting col_counts table")
        
    c.execute('''DELETE FROM col_counts''')
    conn.commit()
    
    for col_name, val in counts:
        print(f"set {col_name}:{val}")
        c.execute('''INSERT INTO col_counts VALUES (?,?)''',[col_name,val])
        
    conn.commit()

list(c.execute('''SELECT count FROM col_counts'''))

[(8,),
 (8,),
 (4698,),
 (7672,),
 (27,),
 (8473,),
 (554,),
 (37,),
 (2662436,),
 (6668832,),
 (8186,),
 (6,),
 (5,),
 (2626,),
 (9,),
 (10,),
 (436,),
 (5,),
 (69,),
 (173,),
 (61,)]

In [27]:
conn.commit()
conn.close()

In [49]:
c.execute("SELECT COUNT(*) FROM data_cleaned").fetchone()[0]

40400000

In [52]:
c.execute("SELECT * FROM data_cleaned where rowid = 10200000").fetchone()

(7068516175296651002,
 0,
 1.791759469228055,
 1,
 2,
 35,
 32,
 3,
 1,
 1,
 1,
 1,
 101212,
 48,
 1,
 1,
 953,
 1,
 1,
 207,
 1,
 1,
 23,
 38)

In [117]:

for i in range(24):
    o = o +"?,"
print(o)

cat_dicts[col_names[0]][(1005,)]

?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,


1

In [105]:
len(cat_dicts["device_id"])

2662436

In [63]:
cc = "device_ip"
uniq_cid = c.execute(f"SELECT DISTINCT {cc} FROM data")
m = {}
counter = 1
for item in uniq_cid:
    m[item] = counter
    counter = counter + 1
print(len(m))

6668831


In [76]:
1.000037190421512e+19

1.000037190421512e+19

In [37]:
import datetime
import math

In [44]:
math.log(4 + 1)

1.6094379124341003

In [77]:
c.close()

In [78]:
conn.close()

In [26]:
cc = list(c.execute('''SELECT count FROM col_counts'''))
cc

[(8,),
 (8,),
 (4698,),
 (7672,),
 (27,),
 (8473,),
 (554,),
 (37,),
 (2662436,),
 (6668832,),
 (8186,),
 (6,),
 (5,),
 (2626,),
 (9,),
 (10,),
 (436,),
 (5,),
 (69,),
 (173,),
 (61,)]

In [25]:
#100,1000,10000
log_bins = [0,0,0,0,0,0,0,0]
for v in cc:
    idx = math.ceil(max(math.log10(v[0]) - 2,0))
    log_bins[idx] = log_bins[idx] + 1
    
log_bins

[11, 3, 5, 0, 0, 2, 0, 0]

2